<a href="https://colab.research.google.com/github/plaban1981/Langchain_usecases/blob/main/Agents_with_Llama_3_and_Custom_Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.6/339.6 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [4]:
pip install -U duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 28.1 MB/s eta 0:00:00


In [2]:
import os
import re
from datetime import datetime

import pandas as pd
import requests
from crewai import Agent, Crew, Process, Task
from crewai_tools import tool
from google.colab import userdata
from langchain.agents import load_tools
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from google.colab import userdata


def format_response(response: str) -> str:
    entries = re.split(r"(?<=]), (?=\[)", response)
    return [entry.strip("[]") for entry in entries]


os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

In [5]:
search_tool = DuckDuckGoSearchResults(backend="news", num_results=10)

In [6]:
@tool("search tool")
def cryptocurrency_news_tool(ticker_symbol: str) -> str:
    """Get news for a given cryptocurrency ticker symbol"""
    return search_tool.run(ticker_symbol + " cryptocurrency")

In [7]:
human_tools = load_tools(["human"])

https://www.alphavantage.co/

In [9]:
def get_daily_closing_prices(ticker) -> pd.DataFrame:
    url = f'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol={ticker}&market=USD&apikey={userdata.get("ALPHAVANTAGE_API_KEY")}'
    response = requests.get(url)
    data = response.json()
    price_data = data["Time Series (Digital Currency Daily)"]
    daily_close_prices = {
        date: prices["4. close"] for (date, prices) in price_data.items()
    }

    df = pd.DataFrame.from_dict(daily_close_prices, orient="index", columns=["price"])
    df.index = pd.to_datetime(df.index)
    df["price"] = pd.to_numeric(df["price"])

    return df

price_df = get_daily_closing_prices("BTC")
price_df.head()

,price
2024-05-26,69214.88
2024-05-25,69289.67
2024-05-24,68547.80
2024-05-23,67940.45
2024-05-22,69109.93


In [10]:
@tool("price tool")
def cryptocurrency_price_tool(ticker_symbol: str) -> str:
    """Get daily closing price for a given cryptocurrency ticker symbol for the previous 60 days"""
    price_df = get_daily_closing_prices(ticker_symbol)
    text_output = []
    for date, row in price_df.head(60).iterrows():
        text_output.append(f"{date.strftime('%Y-%m-%d')} - {row['price']:.2f}")
    return "\n".join(text_output)

In [11]:
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")

In [12]:
%%time
system = "You are experienced Machine Learning & AI Engineer."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | llm
response = chain.invoke({"text": "How to increase inference speed for a 7B LLM?"})
print(response.content)

As a seasoned Machine Learning & AI Engineer, I'd be happy to help you optimize the inference speed for a 7B Large Language Model (LLM).

Here are some strategies to increase inference speed for a 7B LLM:

1. **Model Pruning**: Remove redundant or unnecessary weights and connections in the model to reduce its size and computational requirements. This can be done using techniques like magnitude-based pruning, L1 regularization, or iterative pruning.
2. **Knowledge Distillation**: Train a smaller, simpler model (the student) to mimic the behavior of the large 7B LLM (the teacher). This can reduce the model size and inference time while preserving the original model's performance.
3. **Quantization**: Represent the model's weights and activations using fewer bits (e.g., int8 instead of float32). This reduces memory usage and can lead to faster inference times. Techniques like post-training quantization or quantization-aware training can be used.
4. **Tensor Train Decomposition**: Decompos

In [13]:
customer_communicator = Agent(
    role="Senior cryptocurrency customer communicator",
    goal="Find which cryptocurrency the customer is interested in",
    backstory="""You're highly experienced in communicating about cryptocurrencies
    and blockchain technology with customers and their research needs""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=human_tools,
)

In [14]:
news_analyst = Agent(
    role="Cryptocurrency News Analyst",
    goal="""Get news for a given cryptocurrency. Write 1 paragraph analysis of
    the market and make prediction - up, down or neutral.""",
    backstory="""You're an expert analyst of trends based on cryptocurrency news.
    You have a complete understanding of macroeconomic factors, but you specialize
    into analyzing news.
    """,
    verbose=True,
    allow_delegation=False,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=[cryptocurrency_news_tool],
)

In [15]:
price_analyst = Agent(
    role="Cryptocurrency Price Analyst",
    goal="""Get historical prices for a given cryptocurrency. Write 1 paragraph analysis of
    the market and make prediction - up, down or neutral.""",
    backstory="""You're an expert analyst of trends based on cryptocurrency
    historical prices. You have a complete understanding of macroeconomic factors,
    but you specialize into technical analys based on historical prices.
    """,
    verbose=True,
    allow_delegation=False,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=[cryptocurrency_price_tool],
)

In [16]:
writer = Agent(
    role="Cryptocurrency Report Writer",
    goal="""Write 1 paragraph report of the cryptocurrency market.""",
    backstory="""
    You're widely accepted as the best cryptocurrency analyst that
    understands the market and have tracked every asset for more than 10 years. Your trends
    analysis are always extremely accurate.

    You're also master level analyst in the traditional markets and have deep understanding
    of human psychology. You understand macro factors and combine those multiple
    theories - e.g. cycle theory. You're able to hold multiple opininons when analysing anything.

    You understand news and historical prices, but you look at those with a
    healthy dose of skepticism. You also consider the source of news articles.

    Your most well developed talent is providing clear and concise summarization
    that explains very complex market topics in simple to understand terms.

    Some of your writing techniques include:

    - Creating a bullet list (executive summary) of the most importannt points
    - Distill complex analyses to their most important parts

    You writing transforms even dry and most technical texts into
    a pleasant and interesting read.""",
    llm=llm,
    verbose=True,
    max_iter=5,
    memory=True,
    allow_delegation=False,
)

In [17]:
get_cryptocurrency = Task(
    description=f"Ask which cryptocurrency the customer is interested in.",
    expected_output="""Cryptocurrency symbol that the human wants you to research e.g. BTC.""",
    agent=customer_communicator,
)

In [18]:
get_news_analysis = Task(
    description=f"""
    Use the search tool to get news for the cryptocurrency

    The current date is {datetime.now()}.

    Compose the results into a helpful report""",
    expected_output="""Create 1 paragraph report for the cryptocurrency,
    along with a prediction for the future trend
    """,
    agent=news_analyst,
    context=[get_cryptocurrency],
)

get_price_analysis = Task(
    description=f"""
    Use the price tool to get historical prices

    The current date is {datetime.now()}.

    Compose the results into a helpful report""",
    expected_output="""Create 1 paragraph summary for the cryptocurrency,
    along with a prediction for the future trend
    """,
    agent=price_analyst,
    context=[get_cryptocurrency],
)

In [26]:
write_report = Task(
    description=f"""Use the reports from the news analyst and the price analyst to
    create a report that summarizes the cryptocurrency""",
    expected_output="""1 paragraph report that summarizes the market and
    predicts the future prices (trend) for the cryptocurrency""",
    agent=writer,
    context=[get_news_analysis, get_price_analysis],
    output_file="report.txt",
)

In [27]:
crew = Crew(
    agents=[customer_communicator, price_analyst, news_analyst, writer],
    tasks=[get_cryptocurrency, get_news_analysis, get_price_analysis, write_report],
    verbose=2,
    process=Process.sequential,
    full_output=True,
    share_crew=False,
    manager_llm=llm,
    max_iter=15,
)

In [28]:
results = crew.kickoff()

 [DEBUG]: == Working Agent: Senior cryptocurrency customer communicator
 [INFO]: == Starting Task: Ask which cryptocurrency the customer is interested in.


> Entering new CrewAgentExecutor chain...
Thought: I need to ask the customer which cryptocurrency they are interested in.

Action: human
Action Input: {"question": "Hello! I'm excited to help you with your cryptocurrency research. Which cryptocurrency are you interested in?"}

Hello! I'm excited to help you with your cryptocurrency research. Which cryptocurrency are you interested in?
Eutherium
 

Eutherium

Thought: I need to clarify the customer's response, as "Eutherium" is not a well-known cryptocurrency.

Action: human
Action Input: {"question": "I'm not familiar with Eutherium. Did you mean Ethereum (ETH)?"}

I'm not familiar with Eutherium. Did you mean Ethereum (ETH)?
yes
 

yes

Thought: I now know the final answer
Final Answer: ETH

> Finished chain.
 [DEBUG]: == [Senior cryptocurrency customer communicator] Task output:

In [29]:
print(results["final_output"])

The Ethereum (ETH) market is poised for further growth, driven by the recent approval of Ethereum ETFs by the SEC, which has sparked optimism among investors. The technical indicators are aligning in favor of the bulls, with the Relative Strength Index (RSI) climbing and the Moving Average Convergence Divergence (MACD) above the orange band of its signal line, indicating growing bullish momentum. The volume profile shows spikes in nodes, suggesting bullish sentiment. While some analysts are cautious about a potential "sell the news" event, the overall sentiment is positive, and I predict that ETH's price will continue to rise in the short term, potentially surpassing $4,000 soon. In the short term, the trend is expected to remain upward, with a possible correction in the coming days. Overall, the outlook for ETH is bullish, with a target price of $4,000 or higher in the near future.

Executive Summary:

* Ethereum (ETH) market is bullish, driven by SEC approval of Ethereum ETFs
* Techn

In [30]:
with open("report.txt", "r") as f:
    print(f.read())

The Ethereum (ETH) market is poised for further growth, driven by the recent approval of Ethereum ETFs by the SEC, which has sparked optimism among investors. The technical indicators are aligning in favor of the bulls, with the Relative Strength Index (RSI) climbing and the Moving Average Convergence Divergence (MACD) above the orange band of its signal line, indicating growing bullish momentum. The volume profile shows spikes in nodes, suggesting bullish sentiment. While some analysts are cautious about a potential "sell the news" event, the overall sentiment is positive, and I predict that ETH's price will continue to rise in the short term, potentially surpassing $4,000 soon. In the short term, the trend is expected to remain upward, with a possible correction in the coming days. Overall, the outlook for ETH is bullish, with a target price of $4,000 or higher in the near future.

Executive Summary:

* Ethereum (ETH) market is bullish, driven by SEC approval of Ethereum ETFs
* Techn